In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import OneClassSVM
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.datasets import make_blobs
from sklearn.metrics import make_scorer, accuracy_score
from scipy.io import loadmat

In [24]:
def fix_mat(data):
    if data.dtype.names:
        new_data = dict()
        for name in data.dtype.names:
            new_data[name]=data[0][name][0]
        for k,v in new_data.items():
            if v.dtype.names:
                new_data[k] = fix_mat(v)
            else:
                new_data[k] = np.squeeze(v)
        return new_data
    else:
        return data

In [25]:
# load data
subject = 'test'
day = '20250108'
path = '/home/paolo/cvsa_ws/src/analysis_cvsa/detectShift/dataset_SVMoneclass.mat'
dataset = loadmat(path)

data = fix_mat(dataset['data'])
pos = fix_mat(dataset['pos'])
typ = fix_mat(dataset['typ'])
dur = fix_mat(dataset['dur'])


In [27]:
fix_pos = pos[typ == 781]
fix_dur = dur[typ == 781]
X_fix_all = data[fix_pos[0]:fix_pos[0] + fix_dur[0]]
for i in range(1, fix_pos.shape[0]):
    temp_data = data[fix_pos[i]:fix_pos[i] + fix_dur[i]]
    X_fix_all = np.vstack((X_fix_all, temp_data))
    
X_train = X_fix_all[0:X_fix_all.shape[0]//2, :]


In [20]:
# Step 2: Define a custom scoring function for One-Class SVM
def one_class_svm_score(y_true, y_pred):
    """
    Custom scoring function for One-Class SVM.
    Since One-Class SVM labels inliers as 1 and outliers as -1,
    we assume all test samples should be inliers (i.e., 1).
    """
    return accuracy_score(y_true, y_pred)

scorer = make_scorer(one_class_svm_score)

# Step 3: Set up the parameter grid for cross-validation
param_grid = {
    'nu': [0.1, 0.2, 0.40, 0.50],        # Fraction of expected outliers
    'gamma': ['scale', 'auto']      # Kernel coefficient
}

# Step 4: Perform cross-validation using GridSearchCV
clf = GridSearchCV(OneClassSVM(kernel='rbf'), param_grid, cv=5, scoring=scorer,  verbose=3)
y_train = np.ones((X_train.shape[0], 1))
clf.fit(X_train, y_train)

# Step 5: Best parameters and model evaluation
print("Best Parameters:", clf.best_params_)
print("Best Cross-Validation Score:", clf.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ...............gamma=scale, nu=0.1;, score=0.896 total time=   3.9s
[CV 2/5] END ...............gamma=scale, nu=0.1;, score=0.906 total time=   3.8s
[CV 3/5] END ...............gamma=scale, nu=0.1;, score=0.906 total time=   3.6s
[CV 4/5] END ...............gamma=scale, nu=0.1;, score=0.895 total time=   3.6s
[CV 5/5] END ...............gamma=scale, nu=0.1;, score=0.893 total time=   3.6s
[CV 1/5] END ...............gamma=scale, nu=0.2;, score=0.786 total time=   8.7s
[CV 2/5] END ...............gamma=scale, nu=0.2;, score=0.811 total time=   8.5s
[CV 3/5] END ...............gamma=scale, nu=0.2;, score=0.804 total time=   8.5s
[CV 4/5] END ...............gamma=scale, nu=0.2;, score=0.803 total time=   8.6s
[CV 5/5] END ...............gamma=scale, nu=0.2;, score=0.795 total time=   8.6s
[CV 1/5] END ...............gamma=scale, nu=0.4;, score=0.606 total time=  16.4s
[CV 2/5] END ...............gamma=scale, nu=0.4;,

KeyboardInterrupt: 

In [ ]:
model_svm = OneClassSVM(kernel='rbf', nu=clf.best_params_['nu'], gamma=clf.best_params_['gamma'])
model_svm.fit(X_train, y_train)

OneClassSVM(nu=0.1)

In [ ]:
%matplotlib qt

pred = model_svm.predict(data)
 
fig, ax = plt.subplots()

# Plot the rectangle
for i, x in enumerate(baseline_pos):
    x_left = x
    y_left = - 1
    width = baseline_dur[i]
    height = 2
    rect = plt.Rectangle((x_left, y_left), width, height, edgecolor='lightgrey', facecolor='lightgrey', linewidth=2)
    ax.add_patch(rect)
    
    
ax.scatter(range(0, pred.shape[0], 1), pred, s=0.5)
plt.show()

## LOF

In [28]:
from sklearn.neighbors import LocalOutlierFactor
import math 

lof = LocalOutlierFactor(n_neighbors=250, novelty=True)
lof.fit(X_train)

LocalOutlierFactor(n_neighbors=250, novelty=True)

In [29]:
%matplotlib qt

predlof = lof.predict(data)
 
fig, ax = plt.subplots()

# Plot the rectangle
for i, x in enumerate(baseline_pos):
    x_left = x
    y_left = - 1
    width = baseline_dur[i]
    height = 2
    rect = plt.Rectangle((x_left, y_left), width, height, edgecolor='lightgrey', facecolor='lightgrey', linewidth=2)
    ax.add_patch(rect)
    
    
ax.scatter(range(0, predlof.shape[0], 1), predlof, s=0.5)
plt.show()